In [1]:
%loadFromPOM pom.xml
%classpath C:\\Project\\HAPI-fhir-Lab

In [2]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import ca.uhn.fhir.validation.*;
import org.hl7.fhir.r4.model.*;
import org.hl7.fhir.common.hapi.validation.validator.*;
import org.hl7.fhir.common.hapi.validation.support.*;
import ca.uhn.fhir.context.support.*;
import java.nio.file.*;


In [3]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [5]:
// Create a validator and register it with the context
FhirValidator validator = ctx.newValidator();

// Create a validation module and register itwith the validator
//IValidatorModule module = new FhirInstanceValidator(ctx);

// Register the module with the validator
//validator.registerValidatorModule(module);

NpmPackageValidationSupport npmPackageSupport = new NpmPackageValidationSupport(ctx);
npmPackageSupport.loadPackageFromClasspath("classpath:profiles/tw-core-ig-v0.1.1.tgz");

// Create a validation support chain
ValidationSupportChain validationSupportChain = new ValidationSupportChain(
    npmPackageSupport,
    new DefaultProfileValidationSupport(ctx),
    new InMemoryTerminologyServerValidationSupport(ctx),
    new CommonCodeSystemsTerminologyService(ctx),
    new SnapshotGeneratingValidationSupport(ctx));

// Set the chain on the validator
FhirInstanceValidator instanceValidator = new FhirInstanceValidator(validationSupportChain);
validator.registerValidatorModule(instanceValidator);

// read FHIR Data from C:\Project\HAPI-fhir-Lab\data\Patient-pat-example.json
String data = Files.readString(Paths.get("C:/Project/HAPI-fhir-Lab/data/Patient-pat-example.json"));

// serialize to Patient object
Patient pat = ctx.newJsonParser().parseResource(Patient.class, data);

// Validate the resource
ValidationResult result = validator.validateWithResult(pat);

// Print the validation result
if (result.isSuccessful()) {
    System.out.println("Validation passed");
} else {
    System.out.println("Validation failed");
    result.getMessages().forEach(msg -> System.out.println(msg.getLocationString() + " " + msg.getMessage()));
}


Validation failed
Patient Patient.gender: minimum required = 1, but only found 0 (from https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore)


In [6]:
// read FHIR Data from C:\Project\HAPI-fhir-Lab\synthetic-data\fhir\Alejandro916_Espinoza81.json
String json = new String(Files.readAllBytes(Paths.get("C:\\Project\\HAPI-fhir-Lab\\synthetic-data\\fhir\\Alejandro916_Espinoza81.json")));

// Parse the JSON encoded FHIR resource with Bundle class
Bundle bundle = ctx.newJsonParser().parseResource(Bundle.class, json);

// Read Data from Bundle
ValidationResult result = null;
String resourceType = null;
for (Bundle.BundleEntryComponent entry : bundle.getEntry()) {

    if (entry.getResource().getResourceType().name().equals("Patient")) {
        Patient patient = (Patient) entry.getResource();
       // Validate the resource
        result = validator.validateWithResult(patient);
        resourceType = "Patient";
    }
    //else if (entry.getResource().getResourceType().name().equals("Observation")) {
    //    Observation observation = (Observation) entry.getResource();
    //    result = validator.validateWithResult(observation);
    //    resourceType = "Observation";
    //}
    //else if (entry.getResource().getResourceType().name().equals("Encounter")) {
    //    Encounter encounter = (Encounter) entry.getResource();
    //    result = validator.validateWithResult(encounter);
    //    resourceType = "Encounter";
    //}
    else{
        result = null;
    }

    // Print the validation result
    if (result != null) {
        if (result.isSuccessful()) {
            System.out.println("Validation passed for" + " " + resourceType);
        } else {
            System.out.println("Validation failed for" + " " + resourceType);
            result.getMessages().forEach(msg -> System.out.println(msg.getLocationString() + " " + msg.getMessage()));  
        }
    }
}

Validation failed for Patient
Patient.extension[0] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.extension[1] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.extension[2] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.extension[3] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.identifier[0] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.identifier[2] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.